# PRIPOROČILNI SISTEM GLEDE NA ... (To be added)

## CILJI:
    - na podlagi podanih informacij (izbranih žanr) najti filme, ki najbolj ustrezajo tem parametrom

In [32]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math as math
import time 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import MiniBatchKMeans


In [33]:
netflix = pd.read_csv('Netflix_all.csv')

netflix['date_added'] = pd.to_datetime(netflix['date_added'], format="%Y/%m/%d") 
netflix['day'] = netflix['date_added'].dt.day

In [34]:
netflix['directors'] = netflix['director'].apply(lambda l: [] if pd.isna(l) else [i.strip() for i in l.split(",")])
netflix['categories'] = netflix['listed_in'].apply(lambda l: [] if pd.isna(l) else [i.strip() for i in l.split(",")])
netflix['actors'] = netflix['cast'].apply(lambda l: [] if pd.isna(l) else [i.strip() for i in l.split(",")])
netflix['countries'] = netflix['country'].apply(lambda l: [] if pd.isna(l) else [i.strip() for i in l.split(",")])
netflix.head(10)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,...,description,averageRating,numVotes,month_added,year_added,day,directors,categories,actors,countries
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China",2019-09-09,2019,TV-PG,90 min,...,Before planning an awesome wedding for his gra...,3.1,249.0,September,2019,9.0,"[Richard Finn, Tim Maltby]","[Children & Family Movies, Comedies]","[Alan Marriott, Andrew Toth, Brian Dobson, Col...","[United States, India, South Korea, China]"
1,80117401,Movie,Jandino: Whatever it Takes,-1,Jandino Asporaat,United Kingdom,2016-09-09,2016,TV-MA,94 min,...,Jandino Asporaat riffs on the challenges of ra...,5.2,18.0,September,2016,9.0,[-1],[Stand-Up Comedy],[Jandino Asporaat],[United Kingdom]
2,70234439,TV Show,Transformers Prime,-1,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,2018-09-08,2013,TV-Y7-FV,1 Season,...,"With the help of three human allies, the Autob...",7.8,4968.0,September,2018,8.0,[-1],[Kids' TV],"[Peter Cullen, Sumalee Montano, Frank Welker, ...",[United States]
3,80058654,TV Show,Transformers: Robots in Disguise,-1,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,2018-09-08,2016,TV-Y7,1 Season,...,When a prison ship crash unleashes hundreds of...,6.0,774.0,September,2018,8.0,[-1],[Kids' TV],"[Will Friedle, Darren Criss, Constance Zimmer,...",[United States]
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,2017-09-08,2017,TV-14,99 min,...,When nerdy high schooler Dani finally attracts...,5.2,4980.0,September,2017,8.0,[Fernando Lebrija],[Comedies],"[Nesta Cooper, Kate Walsh, John Michael Higgin...",[United States]


In [ ]:
text_content = netflix['description']
vector = TfidfVectorizer(max_df=0.4,         # drop words that occur in more than X percent of documents
                             min_df=1,      # only use words that appear at least X times
                             stop_words='english', # remove stop words
                             lowercase=True, # Convert everything to lower case 
                             use_idf=True,   # Use idf
                             norm=u'l2',     # Normalization
                             smooth_idf=True # Prevents divide-by-zero errors
                            )
tfidf = vector.fit_transform(text_content)